In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *

from torchlensmaker.core.transforms import TransformBase, TranslateTransform

from torchlensmaker.viewer.render_sequence import ForwardArtist
from torchlensmaker.elements.optical_surfaces import CollisionSurface
from torchlensmaker.core.physics import reflection

Tensor = torch.Tensor

class XZPlaneMirrorTransform(TransformBase):
    def direct_points(self, points: Tensor) -> Tensor:
        scale = torch.tensor([1, -1, 1], 
                           dtype=points.dtype, 
                           device=points.device)
        return points * scale

    def inverse_points(self, points: Tensor) -> Tensor:
        return self.direct_points(points)
    
    def direct_vectors(self, vectors: Tensor) -> Tensor:
        scale = torch.tensor([1, -1, 1], 
                dtype=vectors.dtype, 
                device=vectors.device)
        return vectors * scale

    def inverse_vectors(self, vectors: Tensor) -> Tensor:
        return self.direct_vectors(vectors)

    def hom_matrix(self) -> Tensor:
        mat = torch.eye(4, dtype=self.dtype)
        mat[1, 1] = -1  # Flip Y
        return mat


class Primary(nn.Module):
    def __init__(self, surface):
        super().__init__()
        self.surface = surface
        self.collision_surface = CollisionSurface(surface)

    def forward(self, data):
        dim, dtype = data.dim, data.dtype

        t1, normals1, valid1, _ = self.collision_surface(data)
        t2, normals2, valid2, _ = self.collision_surface(data.replace(
            transforms=[XZPlaneMirrorTransform(dim, dtype)] + data.transforms
        ))

        # assert all valid
        #assert torch.all(valid1)
        #assert torch.all(valid2)
        
        # Keep smallest t
        take1 = t1 < t2
        t = torch.where(take1, t1, t2)
        valid = torch.full_like(valid1, True)
        normals = torch.where(take1.unsqueeze(-1).expand_as(normals1), normals1, normals2)

        #t, normals, valid, _ = self.collision_surface(data.replace(
        #    transforms=data.transforms + [TranslateTransform(torch.tensor([500., 0., 0.], dtype=torch.float64))]
        #))
        
        # full frame collision points for both surfaces
        collision_points = data.P + t.unsqueeze(1).expand_as(data.V) * data.V

        # Compute reflection for colliding rays
        reflected = reflection(data.V, normals)

        return data.filter_variables(valid).replace(
            P=collision_points,
            V=reflected,
            #transforms=new_chain,
        )


base_height = 100

# Parameters
A = tlm.parameter(-1.0494)
#K = tlm.parameter(-1.0)
G0 = tlm.parameter(-301.7344)
T = tlm.parameter(38.8971)
R = tlm.parameter(4.3)

# XY polynomial parameters
C = tlm.parameter(torch.zeros((13,5), dtype=torch.float64))
fixed_mask = torch.zeros_like(C, dtype=torch.bool)
fixed_mask[0, 0] = True  # Freeze position (0,0)
C.register_hook(lambda grad: grad.masked_fill(fixed_mask, 0.))

cylinder = tlm.ImplicitCylinder(*torch.tensor([-50/2, 50/2, 37.02/2], dtype=torch.float64).unbind())
rod_data = StoreVar(lambda data: data)

# Primary mirror
sag = tlm.SagSum([
    tlm.Parabolic(A=A, normalize=True),
    tlm.XYPolynomialN(C)
])
primary = tlm.SagSurface(1800, sag)
primary_data = StoreVar(lambda data: data)

plane = tlm.CircularPlane(3000)

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    XXLightSource.load(),

    # Primary mirror
    tlm.Gap(1000-base_height),
    tlm.Translate3D(y=T),
    tlm.Rotate3D(z=R),
    primary_data,
    Primary(primary),

    # Fixed rod
    tlm.AbsolutePosition(x=G0),
    #tlm.Rotate3D(y=45),
    rod_data,
    NonImagingRod(cylinder),
)

from torchlensmaker.viewer.render_sequence import Artist
from torchlensmaker.viewer import tlmviewer

class PrimaryArtist(Artist):
    def render_module(self, collective, module) -> list[Any]:
        # render both surfaces
        inputs = collective.input_tree[module]
        dim, dtype = inputs.dim, inputs.dtype

        tf = tlm.forward_kinematic(inputs.transforms)
        tf2 = tlm.forward_kinematic([XZPlaneMirrorTransform(dim, dtype)] + inputs.transforms)

        return [tlmviewer.render_surface(module.surface, tf, dim), tlmviewer.render_surface(module.surface, tf2, dim)]

    def render_rays(self, collective, module) -> list[Any]:
        return []

xxrender(optics, sampling={"xx": 50, "letter": "both"}, extra_artists={Primary: PrimaryArtist()}, end=200)

In [ ]:
param_groups = [
    {'params': [A], 'lr': 0.0001},
    {'params': [G0], 'lr': 5},
    {'params': [T], 'lr': 1.5},
    #{'params': [R], 'lr': 30},
    {'params': [C], 'lr': 0.00010},
]


record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 5000, "disable_viewer": True, "letter": "positive"},
    dim = 3,
    num_iter = 100,
    nshow=500,
)

plot_record(record, param_groups, optics)
record.best()

print()
print("Final values")
print("A", A)
print("G0", G0)
print("T", T)
print("R", R)

# Print rod position
target = rod_data.value.target()
print("ROD X", target[1].item())
print("ROD Y", target[2].item())
print("ROD Z", -target[0].item())

xxrender(optics, sampling={"xx": 500, "letter": "both"}, extra_artists={Primary: PrimaryArtist()}, end=200)

In [ ]:
# Print rod position
target = rod_data.value.target()
print("ROD POSITION IN XX FRAME")
print("ROD X", target[1].item())
print("ROD Y", target[2].item())
print("ROD Z", -target[0].item())
rod_z = f"z{-target[0].item():.0f}"
print(rod_z)

def xxgridH(N):
    x = np.linspace(-1, 499, N)
    y = np.linspace(-499, 499, N)
    X, Y = np.meshgrid(x, y)
    return np.stack((X, Y), -1).reshape(-1, 2)

with torch.no_grad():
    part_primary = tess_mirror(xxgridH(150), primary_data.value.tf(), primary)

    # flip part around the Y=0 plane to make second part
    part_primary2 = tess_mirror(xxgridH(150), primary_data.value.tf(), primary, flipy=True)

part_sides = makesides(part_primary.vectors.dtype)

mesh.Mesh(np.concatenate([
    part_primary.data,
    part_primary2.data,
    part_sides.data
])).save(f"parabola6-double-{rod_z}.stl")

